In [ ]:
# ===== Install Required Libraries =====
!pip install gradio transformers matplotlib --quiet

# ===== Imports =====
import gradio as gr
import matplotlib.pyplot as plt
import io
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ===== CONFIG =====
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"

# ===== LOAD IBM GRANITE MODEL =====
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# ===== Store User Data =====
user_data = {
    "mode": "Professional",
    "history": [],       # Track all chats
    "monthly_records": []  # Track each month's income & expenses
}

# ===== AI Chat Function =====
def chat_with_ai(message, mode):
    user_data["mode"] = mode
    user_data["history"].append({"role": "user", "content": message})
    # Prepare prompt from history
    prompt = ""
    for msg in user_data["history"]:
        prompt += f"{msg['role'].capitalize()}: {msg['content']}\n"
    prompt += "Bot:"
    output = generator(prompt, max_new_tokens=200, temperature=0.7)
    bot_reply = output[0]["generated_text"].split("Bot:")[-1].strip()
    user_data["history"].append({"role": "bot", "content": bot_reply})
    return bot_reply

# ===== Budget Analysis =====
def budget_analysis(income, food, shopping, transport, others):
    record = {"Income": income, "Expenses": {"Food": food, "Shopping": shopping, "Transport": transport, "Others": others}}
    user_data["monthly_records"].append(record)
    total_exp = sum(record["Expenses"].values())
    savings = income - total_exp
    advice = f"Your total expenses are ₹{total_exp}. You save ₹{savings} monthly."
    if savings < 0:
        advice += " You are overspending! Cut non-essential expenses."
    elif savings < income * 0.2:
        advice += " Savings are low. Try saving at least 20% of your income."
    else:
        advice += " Great! Your savings are healthy."

    # Pie chart
    fig, ax = plt.subplots()
    ax.pie(record["Expenses"].values(), labels=record["Expenses"].keys(), autopct='%1.1f%%')
    ax.set_title("Monthly Expense Breakdown")
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig)
    return advice, buf

# ===== Emergency Fund Planner =====
def emergency_fund(monthly_expense, months_target):
    fund_needed = monthly_expense * months_target
    monthly_save = fund_needed / 12
    return f"You need ₹{fund_needed} for an emergency fund. Save ₹{monthly_save:.2f} monthly to reach it in 1 year."

# ===== Savings Goal Planner =====
def savings_goal(goal_amount, months):
    monthly_save = goal_amount / months
    return f"To reach ₹{goal_amount} in {months} months, save ₹{monthly_save:.2f} per month."

# ===== Investment Guidance =====
def investment_guidance(amount, risk_level):
    if risk_level == "Low":
        suggestion = "Consider FDs, PPF, or government bonds."
    elif risk_level == "Medium":
        suggestion = "Consider index funds, balanced mutual funds."
    else:
        suggestion = "Consider stocks, crypto, or high-growth startups."
    return f"For ₹{amount} with {risk_level} risk tolerance: {suggestion}"

# ===== Gradio Interface =====
with gr.Blocks() as demo:
    gr.Markdown("# 💰 Personal Finance Chatbot with Memory")
    gr.Markdown("Tracks your budget and advice over time!")

    mode = gr.Radio(["Student", "Professional"], label="Select Mode", value="Professional")

    with gr.Tab("💬 Chat with AI"):
        user_input = gr.Textbox(label="Ask your finance question")
        chat_output = gr.Textbox(label="Chatbot Response")
        chat_btn = gr.Button("Ask")
        chat_btn.click(chat_with_ai, inputs=[user_input, mode], outputs=chat_output)

    with gr.Tab("📊 Budget Analysis"):
        income = gr.Number(label="Monthly Income (₹)", value=50000)
        food = gr.Number(label="Food (₹)", value=10000)
        shopping = gr.Number(label="Shopping (₹)", value=5000)
        transport = gr.Number(label="Transport (₹)", value=3000)
        others = gr.Number(label="Others (₹)", value=2000)
        budget_output = gr.Textbox(label="Analysis")
        budget_plot = gr.Image(type="pil", label="Expense Chart")
        budget_btn = gr.Button("Analyze Budget")
        budget_btn.click(budget_analysis, inputs=[income, food, shopping, transport, others], outputs=[budget_output, budget_plot])

    with gr.Tab("🚨 Emergency Fund Planner"):
        monthly_expense = gr.Number(label="Monthly Expense (₹)", value=20000)
        months_target = gr.Number(label="Target Months", value=6)
        ef_output = gr.Textbox(label="Plan")
        ef_btn = gr.Button("Generate Plan")
        ef_btn.click(emergency_fund, inputs=[monthly_expense, months_target], outputs=ef_output)

    with gr.Tab("🎯 Savings Goal"):
        goal_amount = gr.Number(label="Goal Amount (₹)", value=100000)
        months = gr.Number(label="Months to Achieve", value=12)
        sg_output = gr.Textbox(label="Plan")
        sg_btn = gr.Button("Generate Goal Plan")
        sg_btn.click(savings_goal, inputs=[goal_amount, months], outputs=sg_output)

    with gr.Tab("📈 Investment Guidance"):
        inv_amount = gr.Number(label="Amount to Invest (₹)", value=50000)
        risk_level = gr.Radio(["Low", "Medium", "High"], label="Risk Level", value="Medium")
        inv_output = gr.Textbox(label="Guidance")
        inv_btn = gr.Button("Get Guidance")
        inv_btn.click(investment_guidance, inputs=[inv_amount, risk_level], outputs=inv_output)

demo.launch(share=True)